In [2]:
import sys
sys.path.append('../')
sys.path

['/home/andrecosta/sideprojects/transformer/notebooks',
 '/home/andrecosta/anaconda3/lib/python312.zip',
 '/home/andrecosta/anaconda3/lib/python3.12',
 '/home/andrecosta/anaconda3/lib/python3.12/lib-dynload',
 '',
 '/home/andrecosta/anaconda3/lib/python3.12/site-packages',
 '../']

In [48]:
from transformer.loss import SimpleLossCompute
from transformer.generator import Generator
from transformer.smoothing import LabelSmoothing
from transformer.fake_data import data_gen
from transformer.model import make_model

import torch
import torch.nn as nn

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
V = 3
batch_size = 5
n_batches = 3
train_data = list(data_gen(V, batch_size, n_batches))

In [50]:
model = make_model(V, V, N=2)
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
loss_compute = SimpleLossCompute(model.generator, criterion)


In [52]:
batch = train_data[0]
out = model.forward(
    batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
)
print(out.shape)
# out

torch.Size([5, 9, 512])


In [61]:
loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)  # ??
loss, loss_node

(tensor(53.8912, grad_fn=<SumBackward0>),
 tensor(1.1976, grad_fn=<DivBackward0>))

In [72]:
batch.src

tensor([[1, 1, 1, 2, 2, 2, 2, 1, 1, 2],
        [1, 1, 1, 1, 1, 1, 2, 1, 1, 2],
        [1, 2, 1, 2, 1, 1, 2, 1, 1, 2],
        [1, 1, 1, 2, 1, 1, 1, 1, 1, 1],
        [1, 1, 2, 1, 2, 2, 1, 1, 2, 1]])

In [71]:
x = model.generator(out)
batch.src.shape, out.shape, x.shape, batch.tgt_y.shape

(torch.Size([5, 10]),
 torch.Size([5, 9, 512]),
 torch.Size([5, 9, 3]),
 torch.Size([5, 9]))

In [87]:
x = x.contiguous().view(-1, x.size(-1))
target = batch.tgt_y.contiguous().view(-1)

In [75]:
smoothing = 0.1
size = 3

torch.Size([5, 9, 3])

In [97]:
target.data

tensor([1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2,
        1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1])

In [99]:
mask = torch.nonzero(target.data == 0)
mask

tensor([], size=(0, 1), dtype=torch.int64)

In [100]:
true_dist = x.data.clone()
# true_dist.shape
true_dist.fill_(smoothing/(size - 2))
true_dist.scatter_(1, target.data.unsqueeze(1), (1-smoothing))
true_dist[:, 0] = 0
true_dist

tensor([[0.0000, 0.9000, 0.1000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.1000, 0.9000],
        [0.0000, 0.9000, 0.1000],
        [0.0000, 0.9000, 0.1000],
        [0.000

In [103]:
crit = nn.KLDivLoss(reduction="sum")

In [106]:
crit(x,  true_dist.clone().detach())

tensor(38.2559, grad_fn=<SumBackward0>)